In [1]:
import sys
sys.path.append("../")
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%config Completer.use_jedi = False

In [2]:
import simpleland

pygame 2.0.1 (SDL 2.0.14, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
from simpleland import runner

In [4]:
from simpleland.common import Base, register_base_cls

In [5]:
class Test(Base):
    
    def __init__(self):
        super().__init__()
        self.hi = True

In [6]:
register_base_cls(Test)

In [7]:
t = Test()
t.get_snapshot()

hi


{'_type': 'Test', 'data': {'hi': True}}

In [8]:
from simpleland.object import GObject

In [9]:
class Testo(GObject):
    def __init__(self):
        super().__init__()
        self.tt = Test()
        

In [10]:
t0 = Testo()

In [11]:
type(t0.tt)

__main__.Test

In [12]:
issubclass(type(t0.tt),Base)

True

In [14]:
t1 = Testo()

In [17]:
t1.load_snapshot(t0.get_snapshot())

id
position
angle
config_id
created_tick
player_id
shape_group
shape_group
data
last_change
is_deleted
enabled
depth
visible
image_width
image_height
shape_color
_update_position_callback
Skipping snapshotting of:_update_position_callback with value <function GObject.__init__.<locals>.<lambda> at 0x7ff311b88290>
image_id_default
rotate_sprites
image_offset
child_object_ids
Skipping snapshotting of:child_object_ids with value set()
tt
Skipping snapshotting of:tt with value <__main__.Test object at 0x7ff311b93610>


In [16]:
t1.tt

In [13]:
t0.get_snapshot()

id
position
angle
config_id
created_tick
player_id
shape_group
shape_group
data
last_change
is_deleted
enabled
depth
visible
image_width
image_height
shape_color
_update_position_callback
Skipping snapshotting of:_update_position_callback with value <function GObject.__init__.<locals>.<lambda> at 0x7ff311b88290>
image_id_default
rotate_sprites
image_offset
child_object_ids
Skipping snapshotting of:child_object_ids with value set()
tt
tt
hi


{'_type': 'Testo',
 'data': {'id': 1,
  'position': None,
  'angle': 0,
  'config_id': None,
  'created_tick': 0,
  'player_id': None,
  'shape_group': {'data': {}},
  'data': {},
  'last_change': None,
  'is_deleted': False,
  'enabled': True,
  'depth': 2,
  'visible': True,
  'image_width': 80,
  'image_height': 80,
  'shape_color': None,
  'image_id_default': None,
  'rotate_sprites': False,
  'image_offset': <Vector2(0, 0)>,
  'tt': {'_type': 'Test', 'data': {'hi': True}}}}

In [4]:
from simpleland.runner import get_game_def, get_player_def
import threading

import argparse
import json
import logging
import math
import socketserver
import struct
import threading
import lz4.frame
from pyinstrument import Profiler
from simpleland.client import GameClient

from simpleland.config import GameDef, PlayerDefinition, ServerConfig
from simpleland.content import Content
from simpleland.common import StateDecoder, StateEncoder


from simpleland.registry import load_game_content, load_game_def
from simpleland.renderer import Renderer
from simpleland.utils import gen_id
import traceback
from simpleland import gamectx
from simpleland.server import GameUDPServer, UDPHandler 

In [5]:
rawargs = "--game_id=survival_grid --tick_rate=60 --disable_sound --tile_size=16 --enable_server --enable_client".split(" ")

In [6]:
args = runner.get_arguments(rawargs)

In [7]:
if not args.enable_server and not args.enable_client and not args.remote_client:
        args.enable_client = True

if args.enable_server and args.enable_client and args.remote_client:
    print("Error: Server and Remote Client cannot be started from the same process. Please run seperately.")
    exit(1)

game_def = get_game_def(
    game_id=args.game_id,
    enable_server=args.enable_server,
    remote_client=args.remote_client,
    port=args.port,
    tick_rate=args.tick_rate,
    content_overrides = json.loads(args.content_overrides)
)

# Get resolution
if args.enable_client and args.resolution == 'f':
    import pygame
    pygame.init()
    infoObject = pygame.display.Info()
    resolution = (infoObject.current_w, infoObject.current_h)
else:
    res_string = args.resolution.split("x")
    resolution = (int(res_string[0]), int(res_string[1]))

player_def = get_player_def(
    enable_client=args.enable_client,
    client_id=args.client_id,
    remote_client=args.remote_client,
    hostname=args.hostname,
    port=args.port,
    disable_textures=args.disable_textures,
    render_shapes=args.render_shapes,
    resolution=resolution,
    fps=args.fps,
    player_type=args.player_type,
    tile_size=args.tile_size,
    debug_render_bodies = args.debug_render_bodies,
    view_type = args.view_type,
    sound_enabled= not args.disable_sound
)

content: Content = load_game_content(game_def)

gamectx.initialize(
    game_def,
    content=content)

if player_def.client_config.enabled:
    renderer = Renderer(
        config = player_def.renderer_config,
        asset_bundle=content.get_asset_bundle())

    client = GameClient(
        renderer=renderer,
        config=player_def.client_config)

    gamectx.add_local_client(client)

server = None
import signal
import sys
def graceful_exit(signum=None, frame=None):
    print("Shutting down")
    if game_def.server_config.enabled:
        # server.shutdown()
        server.server_close()

    if args.enable_profiler:
        profiler.stop()
        print(profiler.output_text(unicode=True, color=True))
    exit()
signal.signal(signal.SIGINT, graceful_exit)

try:
    if game_def.server_config.enabled:

        server = GameUDPServer(
            conn=(game_def.server_config.hostname, game_def.server_config.port),
            config=game_def.server_config)

        server_thread = threading.Thread(target=server.serve_forever)
        server_thread.daemon = True
        server_thread.start()
        print("Server started at {} port {}".format(game_def.server_config.hostname, game_def.server_config.port))
except (Exception,KeyboardInterrupt) as e:
    print(traceback.format_exc())
    print(e)


Loading Static Layers
Loading Current sector (0, 0)
Loading sector (0, 1)
Loading sector (-1, 1)
Loading sector (-1, 0)
Loading sector (-1, -1)
Loading sector (0, -1)
Loading sector (1, 0)
Loading sector (1, -1)
Loading sector (1, 1)
Server started at 0.0.0.0 port 10001


In [8]:
done = True


In [15]:
def dict_compare(d1, d2):
    d1_keys = set(d1.keys())
    d2_keys = set(d2.keys())
    shared_keys = d1_keys.intersection(d2_keys)
    added = d1_keys - d2_keys
    removed = d2_keys - d1_keys
    modified = {o : (d1[o], d2[o]) for o in shared_keys if d1[o] != d2[o]}
    same = set(o for o in shared_keys if d1[o] == d2[o])
    return added, removed, modified, same

x = dict(a=1, b=2)
y = dict(a=2, b=2)
added, removed, modified, same = dict_compare(x, y)

In [16]:
import time

In [17]:
# How many milliseconds?
start_time = time.time()
for i in range(10):
    snap=gamectx.object_manager.get_snapshot_full()
(time.time()-start_time) * 1000

6.987333297729492

In [18]:
o = gamectx.object_manager.get_objects()[2]

In [19]:
snap[0]

{'_type': 'Rock',
 'data': {'id': 2,
  'position': <Vector2(32, 0)>,
  'angle': 0,
  'config_id': 'perm_stone1',
  'created_tick': 0,
  'player_id': None,
  'shape_group': {'data': {'default': {'_type': 'Polygon',
     'data': {'label': 'default',
      'object_id': 2,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 0},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}}}},
  'data': {},
  'last_change': 0,
  'is_deleted': False,
  'enabled': True,
  'depth': 1,
  'visible': True,
  'image_width': 16,
  'image_height': 16,
  'shape_color': None,
  'image_id_default': None,
  'rotate_sprites': False,
  'image_offset': <Vector2(0, 0)>,
  'config': {'collectable': False,
   'pushable': True,
   'count_max': 1,
   'permanent': True,
   'health': 100,
   'remove_on_destroy': True,
   'collision_type': 1,
   'count': 1,
   'default_a

In [20]:
from simpleland.common import StateDecoder,StateEncoder

In [21]:
d1 = snap[0]

In [22]:
results = []
for obj_data in snap:
    obj = gamectx.new_object_from_dict(obj_data)
    obj2_data = obj.get_snapshot()
    result=dict_compare(obj_data,obj2_data)
    results.append(result)
    
    break
    

In [23]:
import pprint

In [24]:
pprint.pformat(obj_data,compact=True) == pprint.pformat(obj2_data,compact=True)

False

In [27]:
pprint.pformat(obj_data,compact=True)

"{'_type': 'Rock',\n 'data': {'_action': {'blocking': True,\n                      'start_tick': 0,\n                      'step_size': 1,\n                      'ticks': 1,\n                      'type': 'spawn'},\n          '_effects': {},\n          '_types': None,\n          'angle': 0,\n          'collectable': False,\n          'collision_type': 1,\n          'config': {'collectable': False,\n                     'collision_type': 1,\n                     'count': 1,\n                     'count_max': 1,\n                     'default_action': 'idle',\n                     'health': 100,\n                     'permanent': True,\n                     'pushable': True,\n                     'remove_on_destroy': True},\n          'config_id': 'perm_stone1',\n          'count': 1,\n          'count_max': 1,\n          'created_tick': 0,\n          'data': {},\n          'default_action_type': 'idle',\n          'depth': 1,\n          'enabled': True,\n          'health': 50,\n       

In [28]:
pprint.pformat(obj2_data,compact=True)

"{'_type': 'Rock',\n 'data': {'_action': {'blocking': True,\n                      'start_tick': 0,\n                      'step_size': 1,\n                      'ticks': 1,\n                      'type': 'spawn'},\n          '_effects': {},\n          '_types': None,\n          'angle': 0,\n          'collectable': False,\n          'collision_type': 1,\n          'config': {'collectable': False,\n                     'collision_type': 1,\n                     'count': 1,\n                     'count_max': 1,\n                     'default_action': 'idle',\n                     'health': 100,\n                     'permanent': True,\n                     'pushable': True,\n                     'remove_on_destroy': True},\n          'config_id': 'perm_stone1',\n          'count': 1,\n          'count_max': 1,\n          'created_tick': 0,\n          'data': {},\n          'default_action_type': 'idle',\n          'depth': 1,\n          'enabled': True,\n          'health': 50,\n       

In [36]:
o = gamectx.new_object_from_dict(obj_data)

In [38]:
o.shape_group

In [39]:
obj2_data

{'_type': 'Rock',
 'data': {'id': 2,
  'position': <Vector2(32, 0)>,
  'angle': 0,
  'config_id': 'perm_stone1',
  'created_tick': 0,
  'player_id': None,
  'shape_group': {'data': {'default': {'_type': 'Polygon',
     'data': {'label': 'default',
      'object_id': 2,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 0},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}}}},
  'data': {},
  'last_change': 0,
  'is_deleted': False,
  'enabled': True,
  'depth': 1,
  'visible': True,
  'image_width': 16,
  'image_height': 16,
  'shape_color': None,
  'image_id_default': None,
  'rotate_sprites': False,
  'image_offset': <Vector2(0, 0)>,
  'config': {'collectable': False,
   'pushable': True,
   'count_max': 1,
   'permanent': True,
   'health': 100,
   'remove_on_destroy': True,
   'collision_type': 1,
   'count': 1,
   'default_a

In [28]:
obj.shape_group.get_snapshot()

{'data': {62: {'_type': 'Polygon',
   'data': {'object_id': 2,
    'id': 62,
    'label': None,
    'vertices': [<Vector2(-8, -8)>,
     <Vector2(-8, 8)>,
     <Vector2(8, 8)>,
     <Vector2(8, -8)>],
    'collision_type': 0},
   'params': {'vertices': [<Vector2(-8, -8)>,
     <Vector2(-8, 8)>,
     <Vector2(8, 8)>,
     <Vector2(8, -8)>]}},
  3: {'_type': 'Polygon',
   'data': {'object_id': 2,
    'id': 3,
    'label': None,
    'vertices': [<Vector2(-8, -8)>,
     <Vector2(-8, 8)>,
     <Vector2(8, 8)>,
     <Vector2(8, -8)>],
    'collision_type': 1,
    'sensor': True},
   'params': {'vertices': [<Vector2(-8, -8)>,
     <Vector2(-8, 8)>,
     <Vector2(8, 8)>,
     <Vector2(8, -8)>]}}},
 '_type': 'ShapeGroup'}

In [26]:
from simpleland.object import GObject
objGObject()

<simpleland.object.GObject object at 0x7fdebc27be90>, id:61, data:{}, dict_data:{'id': 61, 'position': None, 'angle': 0, 'config_id': None, 'created_tick': 0, 'player_id': None, 'shape_group': <simpleland.common.ShapeGroup object at 0x7fdebc27bcd0>, 'data': {}, 'last_change': None, 'is_deleted': False, 'enabled': True, 'depth': 2, 'visible': True, 'image_width': 80, 'image_height': 80, 'shape_color': None, '_update_position_callback': <function GObject.__init__.<locals>.<lambda> at 0x7fde7f74a440>, 'image_id_default': None, 'rotate_sprites': False, 'image_offset': <Vector2(0, 0)>, 'child_object_ids': set()}

In [23]:
obj.get_snapshot()

{'_type': 'Rock',
 'data': {'id': 2,
  'position': <Vector2(32, 0)>,
  'angle': 0,
  'config_id': 'perm_stone1',
  'created_tick': 0,
  'player_id': None,
  'shape_group': {'data': {59: {'_type': 'Polygon',
     'data': {'object_id': 2,
      'id': 59,
      'label': None,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 0},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}},
    3: {'_type': 'Polygon',
     'data': {'object_id': 2,
      'id': 3,
      'label': None,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 1,
      'sensor': True},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}}},
   '_type': 'ShapeGroup'},
  'data': {},
  'last_change': 0,
  'is_deleted': 

In [22]:
obj_data

{'_type': 'Rock',
 'data': {'id': 2,
  'position': <Vector2(32, 0)>,
  'angle': 0,
  'config_id': 'perm_stone1',
  'created_tick': 0,
  'player_id': None,
  'shape_group': {'data': {3: {'_type': 'Polygon',
     'data': {'object_id': 2,
      'id': 3,
      'label': None,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 0},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}}},
   '_type': 'ShapeGroup'},
  'data': {},
  'last_change': 0,
  'is_deleted': False,
  'enabled': True,
  'depth': 1,
  'visible': True,
  'image_width': 16,
  'image_height': 16,
  'shape_color': None,
  'image_id_default': None,
  'rotate_sprites': False,
  'image_offset': <Vector2(0, 0)>,
  'config': {'collectable': False,
   'pushable': True,
   'count_max': 1,
   'permanent': True,
   'health': 100,
   'remove_on_destroy': True,
   'collision_type': 1,


In [21]:
obj.get_snapshot()

{'_type': 'Rock',
 'data': {'id': 2,
  'position': <Vector2(32, 0)>,
  'angle': 0,
  'config_id': 'perm_stone1',
  'created_tick': 0,
  'player_id': None,
  'shape_group': {'data': {59: {'_type': 'Polygon',
     'data': {'object_id': 2,
      'id': 59,
      'label': None,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 0},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}},
    3: {'_type': 'Polygon',
     'data': {'object_id': 2,
      'id': 3,
      'label': None,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 1,
      'sensor': True},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}}},
   '_type': 'ShapeGroup'},
  'data': {},
  'last_change': 0,
  'is_deleted': 

In [18]:
obj2_data

{'_type': 'Rock',
 'data': {'id': 2,
  'position': <Vector2(32, 0)>,
  'angle': 0,
  'config_id': 'perm_stone1',
  'created_tick': 0,
  'player_id': None,
  'shape_group': {'data': {59: {'_type': 'Polygon',
     'data': {'object_id': 2,
      'id': 59,
      'label': None,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 0},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}},
    3: {'_type': 'Polygon',
     'data': {'object_id': 2,
      'id': 3,
      'label': None,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 1,
      'sensor': True},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}}},
   '_type': 'ShapeGroup'},
  'data': {},
  'last_change': 0,
  'is_deleted': 

In [ ]:
obj_data

In [20]:
results

[(set(),
  set(),
  {'data': ({'id': 2,
     'position': <Vector2(32, 0)>,
     'angle': 0,
     'config_id': 'perm_stone1',
     'created_tick': 0,
     'player_id': None,
     'shape_group': {'data': {3: {'_type': 'Polygon',
        'data': {'object_id': 2,
         'id': 3,
         'label': None,
         'vertices': [<Vector2(-8, -8)>,
          <Vector2(-8, 8)>,
          <Vector2(8, 8)>,
          <Vector2(8, -8)>],
         'collision_type': 0},
        'params': {'vertices': [<Vector2(-8, -8)>,
          <Vector2(-8, 8)>,
          <Vector2(8, 8)>,
          <Vector2(8, -8)>]}}},
      '_type': 'ShapeGroup'},
     'data': {},
     'last_change': 0,
     'is_deleted': False,
     'enabled': True,
     'depth': 1,
     'visible': True,
     'image_width': 16,
     'image_height': 16,
     'shape_color': None,
     'image_id_default': None,
     'rotate_sprites': False,
     'image_offset': <Vector2(0, 0)>,
     'config': {'collectable': False,
      'pushable': True,
      'coun

In [27]:
obj_data

{'_type': 'Rock',
 'data': {'id': 2,
  'position': <Vector2(32, 0)>,
  'angle': 0,
  'config_id': 'perm_stone1',
  'created_tick': 0,
  'player_id': None,
  'shape_group': {'data': {3: {'_type': 'Polygon',
     'data': {'object_id': 2,
      'id': 3,
      'label': None,
      'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>],
      'collision_type': 0},
     'params': {'vertices': [<Vector2(-8, -8)>,
       <Vector2(-8, 8)>,
       <Vector2(8, 8)>,
       <Vector2(8, -8)>]}}},
   '_type': 'ShapeGroup'},
  'last_change': 0,
  'is_deleted': False,
  'enabled': True,
  'depth': 1,
  'visible': True,
  'image_width': 16,
  'image_height': 16,
  'shape_color': None,
  'image_id_default': None,
  'rotate_sprites': False,
  'image_offset': <Vector2(0, 0)>,
  'type': 'rock',
  '_types': None,
  'health': 50,
  'permanent': True,
  'remove_on_destroy': True,
  'pushable': True,
  'collision_type': 1,
  'height': 1,
  'collectable': False,

In [21]:
from simpleland.object_manager import GObjectManager

In [22]:
om = GObjectManager()

In [ ]:
om.

In [10]:
while gamectx.state == "RUNNING": 
    if done:
        gamectx.content.reset()
    gamectx.process_client_step()
    gamectx.run_step()
    gamectx.render_client_step()
    for player in gamectx.player_manager.players_map.values():
        # print(f"Player: {player.get_id()} {player.get_object_id()}")
        observation, reward, done, info = gamectx.content.get_step_info(player)
    if gamectx.config.wait_for_user_input:
        # print(f"Player: {player.get_id()} {player.get_object_id()}")
        # print(observation)
        print(reward)
        print(done)
        print(info)
        print("----------")


    gamectx.wait_for_input()

Loading sector (0.0, -2.0)
no sector found at (0.0, -2.0)
Creating sector (0.0, -2.0)
Adding obj at in sector: (0.0, -2.0) at (0.0, -128.0)
---------
---------
Loading sector (1.0, -2.0)
no sector found at (1.0, -2.0)
Creating sector (1.0, -2.0)
Adding obj at in sector: (1.0, -2.0) at (64.0, -128.0)
---------
---------
Loading sector (-1.0, -2.0)
no sector found at (-1.0, -2.0)
Creating sector (-1.0, -2.0)
Adding obj at in sector: (-1.0, -2.0) at (-64.0, -128.0)
---------
---------


SystemExit: 

/home/brandyn/anaconda3/envs/simpleland/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
